In [1]:
import sys
sys.path.extend(['D:\\Documents\\GitHub\\pq-analyzer\\env', 'D:\\Documents\\GitHub\\pq-analyzer\\env\\lib\\site-packages'])

import pandas as pd
import numpy as np
import pickle
from ast import literal_eval

from pdb import set_trace as st
import string
import re
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = stopwords.words('english')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

from IPython.core.interactiveshell import InteractiveShell
import sys
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
np.set_printoptions(threshold = sys.maxsize)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thisi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\thisi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thisi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('../pq-scraper/pqs.csv', sep='|')
for key in ['asker_parliaments', 'askees', 'askees_portfolios', 'responses']:
    df[key] = df[key].apply(literal_eval)
print('Number of rows:', len(df))
assert isinstance(df.loc[0].responses, list)
df.dtypes
#df.head(2)

Number of rows: 19801


asker_name           object
asker_party          object
asker_parliaments    object
askees               object
askees_portfolios    object
question             object
parliament_no         int64
session_no            int64
volume_no             int64
sitting_no            int64
sitting_date         object
report_section       object
title                object
responses            object
is_pq                  bool
dtype: object

In [3]:
gin_stopwords = ['about', 'actually', 'ago', 'agree', 'already', 'also', 'always',
                 'am', 'because', 'bring', 'can', 'cannot', 'cant',
                 'could', 'couldnt', 'did', 'didnt', 'different', 'dont',
                 'eg', 'ei', 'etc', 'even', 'every', 'everywhere', 'example', 'feel', 'felt', 'get',
                 'go', 'good', 'hence', 'however', 'ie', 'im', 'important', 
                 'know', 'like', 'make', 'may', 'maybe', 'mid', 'need', 'no', 'not', 'often', 'oh', 'ok',
                 'one', 'per', 'pre', 'put', 'quite', 'rather', 'really', 'reduce', 'say', 'singapore', 'something', 'su', 'space',
                 'still', 'sum', 'th', 'that', 'thats', 'they', 'theyre', 'theyve',
                 'think', 'three', 'total', 'try', 'us', 'use', 've', 'via', 'wa', 'want', 'way', 'we',
                 'well', 'were', 'weve', 'whatsoever', 'whether', 'wont',
                 'would', 'wouldnt', 'yall', 'yea', 'yeah', 'yes', 'yet',
                 'youd', 'youll', 'youre', 'youve', 'yr']

honorific_stopwords = ['mrs', 'mr', 'ms', 'miss', 'mdm', 'madam', 'mister', 'er', 'assoc', 'prof', 'dr', 'bg']

number_stopwords = ['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth', 'last', 'ii', 'iii', 'iv', 'vi', 'vii', 'viii', 'ix', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty', 'thirty', 'forty', 'fifty', 'sixty', 'seventy', 'eighty', 'ninety', 'hundred', 'thousand', 'million', 'billion', 'trillion']

# "ministry" is a stopword because, if someone asks about ministry of education, then he's interested in education. so we dunnid the "ministry".
hansard_stopwords = ['speaker', 'leader', 'chairman', 'chairwoman', 'chair', 'deputy', 'beg', 'permission', 'move', 'withdraw', 'allocate', 'head', 'cut', 'thank', 'member', 'honorable', 'question', 'clarification', 'grateful', 'year', 'month', 'day', 'reply', 'many', 'ministry', 'minister', 'consider', 'take', 'ask', 'sir', 'maam', 'since', 'especially', 'reason', 'see', 'non', 'due', 'mention', 'respectively', 'respective', 'must', 'expect', 'mean', 'exist', 'cause', 'hon']

adverb_stopwords = ['annually', 'currently', 'early', 'finally', 'firstly', 'fully', 'increasingly', 'likely', 'monthly', 'particularly', 'previously', 'secondly', 'specifically']

add_stopwords = gin_stopwords + honorific_stopwords + hansard_stopwords + number_stopwords + adverb_stopwords
stop_words.extend(add_stopwords)

In [4]:
# Saving stopwords and phrases into files
with open('stopwords.pkl', 'wb') as f:
    pickle.dump(stop_words, f)

## 0. Functions

In [5]:
words_to_sub = {
    'ministry of communications and information': 'mci',
    'ministry of culture community and youth': 'mccy',
    'ministry of defense': 'mindef',
    'ministry of education': 'moe',
    'ministry of finance': 'mof',
    'ministry of foreign affairs': 'mfa', 
    'ministry of health': 'moh',
    'ministry of home affairs': 'mha',
    'ministry of law': 'minlaw',
    'ministry of manpower': 'mom',
    'ministry of national development': 'mnd',
    'ministry of social and family development': 'msf',
    'ministry of sustainability and the environment': 'mse',
    'ministry of trade and industry': 'mti',
    'ministry of transport': 'mot',
    'prime ministers office': 'pmo'
}

def lemmatize(lemmatizer, word):
    for pos in ('a', 'n', 'v', 'r', 's'):
        word = lemmatizer.lemmatize(word, pos=pos)
    return word

def clean_text(text, stop_words = stop_words):
    lemmatizer = WordNetLemmatizer()
    
    if text != text: # np.nan
        return np.nan
    
    else:
        text = str(text)
        text = text.lower()
        
        # Replacing symbols
        for to_replace in ('\n', '&gt; ', '&gt', '/', '-'):
            text = text.replace(to_replace, ' ')
        text = re.sub('\s\s+', ' ', text)
        text = re.sub('[^a-z ]', '', text)
        
        text = f' {text} '
        
        for word_to_sub in words_to_sub.keys():
            text = text.replace(f' {word_to_sub} ', f' {words_to_sub[word_to_sub]} ')
        
        # lemmatize and normalize and remove stopwords
        words = [
            lemmatize(lemmatizer, word) 
            for word in text.strip().split(' ') 
            if word != '' and word not in stop_words]

        text = ' '.join(words).strip()

        return text

In [6]:
def str_replace(string, to_replace, replace_value):
    if string != string: # np.nan
        return np.nan
    
    else:
        return string.replace(to_replace, replace_value).strip()

In [7]:
def num_words(string):
    if string != string: # np.nan
        return np.nan
    
    else: 
        words = [word for word in string.split(' ') if word != '']
        
        return len(words)

In [8]:
def obtain_vocab(col, max_df = 0.5, min_df = 0.001, stop_words = stop_words):
    
    col = col.dropna()
    
    vectorizer = TfidfVectorizer(max_df = max_df,
                                 min_df = min_df,
                                 max_features = None,
                                 ngram_range = (1,1), # avoid memory issues
                                 strip_accents = 'ascii',
                                 stop_words = stop_words)
    
    t = vectorizer.fit_transform(col)
    # Version Issue: get_feature_names_out() may not work; try get_feature_names() instead if so
    if 'get_feature_names_out' in dir(vectorizer):
        ngram = vectorizer.get_feature_names_out()
    else:
        ngram = vectorizer.get_feature_names()

    name_vec_df = pd.DataFrame(t.toarray(), columns = ngram)
    ngram_freq = name_vec_df.sum(axis = 0).reset_index()
    ngram_freq.columns = ['word/phrase', 'score']
    
    return ngram_freq

## 1. Preprocessing Text

In [9]:
qn_raw_col = 'question'
res_raw_col = 'responses'

qn_col = 'question_cleaned'
res_col = 'responses_cleaned'

In [10]:
df.insert(loc = 6,
          column = qn_col,
          value = df[qn_raw_col].apply(clean_text))

In [11]:
df.insert(loc = 15,
          column = res_col,
          value = df[res_raw_col].apply(lambda responses: list(map(lambda r: (r[0], r[1], r[2], clean_text(r[3])), responses))))

In [12]:
df['question_len'] = df[qn_raw_col].apply(num_words)
df['response_count'] = df[res_raw_col].apply(len)
df['response_total_len'] = df[res_raw_col].apply(lambda x: sum(list(map(lambda y: num_words(y[3]), x))))

In [13]:
df.head(2)

asker_name                     asker_party asker_parliaments  \
0      Lee Bee Wah           People's Action Party      (11, 12, 13)   
1  K Thanaletchimi  Nominated Member of Parliament             (13,)   

                                                                                                                 askees  \
0  (Senior Minister of State for the Environment and Water Resources, Minister for the Environment and Water Resources)   
1  (Senior Minister of State for the Environment and Water Resources, Minister for the Environment and Water Resources)   

  askees_portfolios  \
0            (MSE,)   
1            (MSE,)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [14]:
df.loc[0]

asker_name                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [15]:
df.to_csv('pqs_processed.csv', index = False, sep='|')

In [16]:
# Obtaining vocabulary for qns and responses
## This gives us a sense of other stopwords we may have missed, or abbreviations that should be expanded. 
qn_vocab = obtain_vocab(df[qn_col], min_df = 0.0001)
res_vocab = obtain_vocab(df[res_col].apply(lambda responses: ' '.join(list(map(lambda response: response[3], responses)))), min_df = 0.0001)

qn_vocab.head(2)
res_vocab.head(2)

qn_vocab.to_csv('qn_vocab.csv', index = False)
res_vocab.to_csv('res_vocab.csv', index = False)

,word/phrase,score
0,aaa,0.280275
1,aar,2.255834


,word/phrase,score
0,aa,3.231504
1,aaa,1.708646


In [17]:
# look for common acronyms we might've missed
qn_vocab['len'] = qn_vocab['word/phrase'].apply(len)
short_words = qn_vocab[qn_vocab['len'] <= 3]
short_words[short_words['score'] > 20].sort_values('score', ascending=False)

,word/phrase,score,len
5303,hdb,242.844936,3
8036,new,207.439023,3
6465,job,180.238593,3
305,age,168.206550,3
7088,low,140.668376,3
8758,pay,115.177702,3
1501,bus,110.806911,3
2641,cpf,109.934438,3
10880,set,106.722917,3
140,act,101.819588,3


In [18]:
# look for common acronyms we might've missed
res_vocab['len'] = res_vocab['word/phrase'].apply(len)
short_words = res_vocab[res_vocab['len'] <= 3]
short_words[short_words['score'] > 30].sort_values('score', ascending=False)

,word/phrase,score,len
12124,new,404.504617,3
9593,job,321.840669,3
7856,hdb,290.663239,3
10593,low,240.287541,3
451,age,225.387671,3
16365,set,188.436313,3
10292,let,183.805634,3
3960,cpf,167.076372,3
1106,art,162.961000,3
2289,bus,162.092425,3


In [19]:
# adverbs are usually useless.
probable_adverbs = qn_vocab[qn_vocab['word/phrase'].apply(lambda w: w[-2:] == 'ly')]
probable_adverbs[probable_adverbs['score'] >= 20]

,word/phrase,score,len
623,apply,71.905411,5
2834,daily,26.440500,5
3768,elderly,92.816866,7
4355,family,188.237460,6
9880,recently,39.686774,8
11832,supply,66.705328,6


# old stuff from gin below.

In [20]:
# # Searching for qns/responses for which cleaned version contains certain words/phrases
# string = 'everywhere'
# df[[qn_raw_col, qn_col]][df[qn_col].str.contains(string, na = False)]
# df[[res_raw_col, res_col]][df[res_col].str.contains(string, na = False)]